In [1]:
###############################################################################################
# 허깅페이스 Tokenizer 예제
# - 참고 : https://huggingface.co/docs/tokenizers/training_from_memory
# - 참고 : https://towardsdatascience.com/training-bpe-wordpiece-and-unigram-tokenizers-from-scratch-using-hugging-face-3dd174850713
#
# xml-roberta-base 모델 : Unigram Tokenizer 사용함.
# - tokenizer.json 파일을 보면 Tokenzier모델 속성을 알수 있음.
#
# [Tokenizer 과정]
#  1. Tokenizer 모델, 트레이너 정의
#  2. Normalizer, Pre-tokenizers, Decoders, Post-processors 등을 설정
#  3. 훈련시작 및 Tokenizer 저장(*pretrained 연동해서 저장)
#  4. (option) Tokenizer된 vocab들을 파일(.txt)로 저장
#  5. (option) PretrainedTokenizer로 변환(*단 unigram으로 된 tokeizer여야만 됨)
###############################################################################################

from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers, processors
from tokenizers.pre_tokenizers import Whitespace, WhitespaceSplit, Metaspace, Punctuation, Digits
from tokenizers.normalizers import Lowercase, NFC, NFD, NFKC, NFKD, Nmt, Normalizer,Precompiled, Replace, Sequence, Strip, StripAccents

#===============================================================================================
# 토크너나이저 모델 종류 (BPE=BPE(GPT2), UNI=Unigram(xml-roberta-base), WL=WordLevel, WP=WordPiece(bert))
model_type = 'WP' # BPE, UNI, WL, WP
vocab_size = 32000  # vocab 크기 정의

# 훈련 말뭉치(*리스트)
# -> ["./test1.txt','./test2.txt'] 리스트로 여러개 파일을 넣을수 있음
corpus_path = ['../../../korpora/kowiki/kowiki-202206-nlp-corpus-mecab.txt']

# out tokenizer 파일 출력 경로
tokenizer_path = './kowiki-2022-nlp-corpus-mecab-WP.json'

# vocab 파일 저장 경로
vocab_out = './kowiki-2022-nlp-corpus-mecab-WP.txt'

# Pretrained 토크너나이저로 변환된 파일들 생성할 폴더
pretrained_tokenizer_folder = './kowiki-2022-nlp-corpus-mecab-WP'
#===============================================================================================

if model_type == 'WL' or model_type=='WP':
    unk_token = "[UNK]" # unk 토큰 정의(예: distilbert 모델일때)
    spl_tokens = ["[CLS]", "[SEP]", "[UNK]", "[MASK]", "[PAD]"] # special 토큰 정의 (예: distilbert 모델일때)
else:
    unk_token = "<unk>"  # unk 토큰 정의(예: xml-roberta-base 모델일때) 
    spl_tokens = ["<s>", "</s>", "<unk>", "<mask>", "<pad>"] # special 토큰 정의 (예: xml-roberta-base 모델일때)
    

In [2]:
###############################################################################################
# 1. Tokenizer 모델, 트레이너 정의
# 2. Normalizer, Pre-tokenizers, Decoders, Post-processors 등을 설정
# => 모델에 따라 변경해 줘야 함.(tokenizer.json 파일을 불러와서 봐야함)
# => API 참조 : https://huggingface.co/docs/tokenizers/main/en/api/normalizers
###############################################################################################

# 모델 타입에 따라 Tokenizer 모델, 트레이너 얻는 함수
def prepare_tokenizer_trainer(modeltype):
    
    # 타입 검사
    tmpmodeltype = ['BPE', 'UNI', 'WL', 'WP'] 
    assert modeltype in tmpmodeltype, f'잘못된 모델타입 입력 입니다.=>modeltype:{modeltype}'
    
    if modeltype == 'BPE': # GPT-2 모델
        
        tokenizer = Tokenizer(models.BPE(unk_token = unk_token))
        trainer = trainers.BpeTrainer(vocab_size=vocab_size, special_tokens = spl_tokens)
        
        # 1> normalizer : 없음

        # 2> pre-tokenizer : ByteLevel - add_prefix_space:False, trim_offsets:true
        tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

        # 3> decoder : ByteLevel - add_prefix_space:true, trim_offsets:true
        tokenizer.decoder = decoders.ByteLevel()

        # 4> post_processor : ByteLevel -add_prefix_space:true, trim_offsets:false
        tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
        
    elif modeltype == 'UNI': # xml-roberta-base 모델
        tokenizer = Tokenizer(models.Unigram())
        trainer = trainers.UnigramTrainer(vocab_size=vocab_size, unk_token= unk_token, special_tokens = spl_tokens)
        
        # 1> normalizer : Precompiled 인데, 어떻게 사용하는지 모름.(*따라서 일단 NFKC() 로 지정함)
        #tokenizer.normalizer = normalizers.Precompiled()
        tokenizer.normalizer = normalizers.NFKC()
        
        # 2> pre-tokenizer : WhitespaceSplit, Metaspace => 여러개를 묶을때 Sequence를 사용함
        tokenizer.pre_tokenizer = pre_tokenizers.Sequence([WhitespaceSplit(), Metaspace()])

        # 3> decoder : Metaspace
        tokenizer.decoder = decoders.Metaspace()

        # 4> post_processor : TemplateProcessing
        tokenizer.post_processor = processors.TemplateProcessing(single="<s> $A </s>", 
                                                                 pair="<s> $A </s> </s> $B:1 </s>:1", 
                                                                 special_tokens=[("<s>", 0), ("</s>", 1)])

    elif modeltype == 'WL': # distilbert 모델
        tokenizer = Tokenizer(models.WordLevel(unk_token = unk_token))
        trainer = trainers.WordLevelTrainer(vocab_size=vocab_size, special_tokens = spl_tokens)
        
        # 1> normalizer : ??      
        tokenizer.normalizer = normalizers.Sequence([NFKC(), Lowercase()])

        # 2> pre-tokenizer : BertPreTokenizer
        tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

        # 3> decoder : WordPiece
        #tokenizer.decoder = decoders.WordPiece()

        # 4> post_processor : TemplateProcessing
        tokenizer.post_processor = processors.BertProcessing(("[CLS]",0), ("[SEP]",1))
        
    elif modeltype == 'WP': # distilbert 모델
        tokenizer = Tokenizer(models.WordPiece(unk_token = unk_token))
        trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens = spl_tokens)
        
        # 1> normalizer : BertNormalizer
        tokenizer.normalizer = normalizers.BertNormalizer(clean_text = True, 
                                                          handle_chinese_chars = True, 
                                                          strip_accents = False, 
                                                          lowercase = False)

        # 2> pre-tokenizer : BertPreTokenizer
        tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

        # 3> decoder : WordPiece
        tokenizer.decoder = decoders.WordPiece()

        # 4> post_processor : TemplateProcessing
        tokenizer.post_processor = processors.TemplateProcessing(single="[CLS] $A [SEP]", 
                                                                 pair="[CLS] $A [SEP] $B:1 [SEP]:1", 
                                                                 special_tokens=[("[CLS]",0), ("[SEP]",1)])
            
    return tokenizer, trainer

# Tokenizer 모델, 트레이너 선택
tokenizer, trainer = prepare_tokenizer_trainer(model_type)

In [3]:
###############################################################################################
# 3. 훈련시작 및 Tokenizer 저장
###############################################################################################

# 훈련 시작 
tokenizer.train(files=corpus_path, trainer=trainer)

# 훈련된 tokenizer 파일로 저장 
tokenizer.save(tokenizer_path)

In [4]:
###############################################################################################
# 4. (option) Tokenizer된 vocab들을 파일(.txt)로 저장
###############################################################################################

# 저장된 tokenizer 파일 불러옴.
tokenizer = Tokenizer.from_file(tokenizer_path)

# tokenizer 테스트 
#text = ["여기는 BERT와 같은 인공지능 모델을 구현하고, simulate 하는 곳입니다.", "두개의 Text 문장 테스트 결과 입니다."]
text = "여기는 BERT와 같은 인공지능 모델을 구현하고, simulate 하는 곳입니다."
output = tokenizer.encode(text)
print(output.tokens)
print(output.ids)
print('\n')
print(f'<unk>:{tokenizer.token_to_id("<unk>")}')
print('\n')      
for token in zip(output.tokens, output.ids):
    print(token)
    
# toeknizer vocab을 id 순번(id_to_token)으로 리스트에 저장함.
vocab = []
vocablen = tokenizer.get_vocab_size() # tokenizer 길이 얻어옴
for i in range(vocablen):
    vocab.append(tokenizer.id_to_token(i))
    
print(vocab[0:20])

# txt 파일로 저장
# -  vocab을 txt 파일로 저장함
from tqdm.notebook import tqdm
with open(vocab_out, 'w', encoding='utf-8') as f:
    for word in tqdm(vocab):
        f.write(word+'\n')
    

['[CLS]', '여기', '##는', 'BE', '##RT', '##와', '같', '##은', '인공지능', '모델', '##을', '구현', '##하', '##고', ',', 'sim', '##ulate', '하', '##는', '곳', '##입', '##니다', '.', '[SEP]']
[0, 20090, 15334, 24589, 30167, 15919, 11760, 15234, 28665, 19848, 15316, 20465, 15375, 15333, 16, 20326, 26867, 14627, 15334, 11841, 15383, 18970, 18, 1]


<unk>:None


('[CLS]', 0)
('여기', 20090)
('##는', 15334)
('BE', 24589)
('##RT', 30167)
('##와', 15919)
('같', 11760)
('##은', 15234)
('인공지능', 28665)
('모델', 19848)
('##을', 15316)
('구현', 20465)
('##하', 15375)
('##고', 15333)
(',', 16)
('sim', 20326)
('##ulate', 26867)
('하', 14627)
('##는', 15334)
('곳', 11841)
('##입', 15383)
('##니다', 18970)
('.', 18)
('[SEP]', 1)
['[CLS]', '[SEP]', '[UNK]', '[MASK]', '[PAD]', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']


  0%|          | 0/32000 [00:00<?, ?it/s]

In [5]:
###############################################################################################
# 5. (option) PretrainedTokenizer로 변환
# - tokenizer.json 파일을 RobertaTokenizerFast 토크너나이저로 변환
###############################################################################################
from transformers import PreTrainedTokenizerFast, DistilBertTokenizerFast, RobertaTokenizerFast, GPT2TokenizerFast, AutoTokenizer
import os

def pretrain_tokenizer(modeltype, special_tokens, out_path):
      # 타입 검사
    tmpmodeltype = ['BPE', 'UNI', 'WL', 'WP'] 
    assert modeltype in tmpmodeltype, f'잘못된 모델타입 입력 입니다.=>modeltype:{modeltype}'
    
    if modeltype == 'BPE': # GPT-2 모델
        pretrain_tokenizer = GPT2TokenizerFast(tokenizer_file = out_path, additional_special_tokens = special_tokens)
    elif modeltype == 'UNI': # xml-roberta-base 모델
        pretrain_tokenizer = RobertaTokenizerFast(tokenizer_file = out_path, additional_special_tokens = special_tokens)
    elif modeltype == 'WL' or modeltype == 'WP': 
        pretrain_tokenizer = DistilBertTokenizerFast(tokenizer_file = tokenizer_path, additional_special_tokens = spl_tokens)
        
    return pretrain_tokenizer

pre_tokenizer = pretrain_tokenizer(modeltype=model_type, special_tokens = spl_tokens, out_path=tokenizer_path)

# PreTrainedTokenizerFast tokenizer 저장
os.makedirs(pretrained_tokenizer_folder, exist_ok=True)
pre_tokenizer.save_pretrained(pretrained_tokenizer_folder)


# PretrainedTokenizer 테스트
# 저장된 PretrainedTokenizer 불러옴
tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_folder, do_lower_case=False)
text = "여기는 BERT와 같은 인공지능 모델을 구현하고, simulate 하는 곳입니다."
out_encode = tokenizer.encode(text)
print(out_encode)

for idx in out_encode:
    print(f'{tokenizer.convert_ids_to_tokens(idx)}:{idx}')

print(tokenizer.decode(out_encode))


[0, 20090, 15334, 24589, 30167, 15919, 11760, 15234, 28665, 19848, 15316, 20465, 15375, 15333, 16, 20326, 26867, 14627, 15334, 11841, 15383, 18970, 18, 1]
[CLS]:0
여기:20090
##는:15334
BE:24589
##RT:30167
##와:15919
같:11760
##은:15234
인공지능:28665
모델:19848
##을:15316
구현:20465
##하:15375
##고:15333
,:16
sim:20326
##ulate:26867
하:14627
##는:15334
곳:11841
##입:15383
##니다:18970
.:18
[SEP]:1
[CLS] 여기는 BERT와 같은 인공지능 모델을 구현하고, simulate 하는 곳입니다. [SEP]
